In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
import pickle
import torch
import os
import copy
import torch.nn as nn
import seaborn as sns


In [5]:
# Plots
import matplotlib.pyplot as plt

def plot_side_by_side(image1, image2, idx, save_path, titles=None):
    """
    Plots three images side by side.

    Args:
        image1 (np.ndarray): The first image.
        image2 (np.ndarray): The second image.
        image3 (np.ndarray): The third image.
        titles (list, optional): A list of titles for the images.

    Returns:
        None
    """

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    axes[0].set_title(titles[0])
    axes[0].imshow(image1)
    axes[0].axis('off')
    axes[1].set_title(titles[1])
    axes[1].imshow(image2)
    axes[1].axis('off')
    
    plt.tight_layout()
    
def lineplot_acc_perturbation(x, y1,y2, title, xlabel, ylabel, config, base_path):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x, y1, label='Accuracy - Layer-wise Regularized')
    plt.plot(x, y2, label='Accuracy - Regularized')
    plt.legend(loc='upper right')        
    filename = os.path.join(base_path, config + ".jpg")
    plt.savefig(filename, bbox_inches='tight')
    

def lineplot_l2radius_perturbation(x, y1, y2, title, xlabel, ylabel, config, base_path):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x, y1, label="Layer-wise Regularized")
    plt.plot(x, y2, label="Regularized")
    plt.legend(loc='upper left')
    filename = os.path.join(base_path, config + ".jpg")
    plt.savefig(filename, bbox_inches='tight')
    

def plot_histplot(df, color, label, title, ylim, xlim, xlabel, ylabel, base_path, plot_config, auto_bins=False):
    df = df.copy()
    unique_bins = len(df[label].dropna().unique())
    if auto_bins:
        ax = sns.histplot(data=df, x=label, color=color, label=label, bins='auto')
    else:
        ax = sns.histplot(data=df, x=label, color=color, label=label, bins=unique_bins)
    ax.set_ylim(0, ylim)
    ax.set_xlim(0, xlim)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, rotation=90)
    ax.set(ylabel=ylabel)
    plt.title(title)
    ax.legend()
    ax.set(xlabel=xlabel)
    out_path = os.path.join(base_path, plot_config)
    plt.savefig(out_path, bbox_inches='tight')
    plt.close()



# New Attack

In [3]:
all_base_paths = ["/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_f9c2570b43d17cd3f46b942f31d6fb03",
                  ]

MSELoss_none = nn.MSELoss(reduction='none')
MSELoss = nn.MSELoss(reduction='mean')
flatten = nn.Flatten()
num_samples = 2560
og_num_samples = 2560
detector = 'Regular'
c_base=1.0
d_base=0.0
all_eps_list=[
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04]
            ]
plot_path = os.path.join(all_base_paths[0], "Plots")


In [ ]:
plot_path

In [ ]:

all_freq_lists = []
all_theoretical_eps_per_channel = []
all_mse_loss_per_channel = []
all_mse_loss_per_pixel = []
all_mse_max_per_pixel = []
all_mse_loss_per_image = []
all_mse_loss_per_channel_max = []




for eps_idx, base_path in enumerate(all_base_paths):
    if "MT_Baseline_600547deae5b3d506d5c0befd9134c77" in base_path:
        num_samples = 1500
        c_base=1.0

    all_flipped_indices = []
    lst_theoretical_eps_per_channel = []
    lst_mse_loss_per_channel = []
    lst_mse_loss_per_pixel = []
    lst_mse_max_per_pixel = []
    lst_mse_loss_per_image = []
    lst_mse_loss_per_channel_max = []

    prev_flipped_indices = set()
    for eps in (all_eps_list[eps_idx]):
        images_cw_base = f"Adversarial_Datasets/CW_adv_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        images_benign = f"Benign_Datasets/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        predictions_base = f"Predictions/Model/CW_type-adversarial_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        filtered_indices_base = f"Predictions/Perturbed_Samples/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pt"
        images_cw_base = os.path.join(base_path, images_cw_base)
        images_benign = os.path.join(base_path, images_benign)
        predictions_base_path = os.path.join(base_path, predictions_base)
        filtered_indices_base_path = os.path.join(base_path, filtered_indices_base)

        # Filter by common indices
        filtered_indices_base = torch.load(filtered_indices_base_path)
        filtered_indices_base = [x.item() for x in filtered_indices_base]
        curr_flipped_indices = set(filtered_indices_base)
        curr_unique_flipped_indices = curr_flipped_indices.difference(prev_flipped_indices)    
        prev_flipped_indices.update(curr_unique_flipped_indices)
        curr_unique_flipped_indices = list(curr_unique_flipped_indices)
        all_flipped_indices.append(filtered_indices_base)

        with open(images_cw_base, 'rb') as in_file:
            images_cw_base = pickle.load(in_file)
            images_cw_base[0] = images_cw_base[0][curr_unique_flipped_indices]
            images_cw_base[1] = images_cw_base[1][curr_unique_flipped_indices]
            
        with open(images_benign, 'rb') as in_file:
            images_benign = pickle.load(in_file)
            images_benign[0] = images_benign[0][curr_unique_flipped_indices]
            images_benign[1] = images_benign[1][curr_unique_flipped_indices]

        with open(predictions_base_path, 'rb') as in_file:
            predictions = pickle.load(in_file)
            true_labels_base = np.asarray(predictions['true_labels'])[curr_unique_flipped_indices]
            pred_labels_base = np.asarray(predictions['pred_labels'])[curr_unique_flipped_indices]

        mse_loss_per_channel = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_channel = torch.sqrt(mse_loss_per_channel)
        mse_loss_per_channel = mse_loss_per_channel.mean(dim=1)

        mse_loss_per_channel_max = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_channel_max = torch.sqrt(mse_loss_per_channel_max)
        mse_loss_per_channel_max = mse_loss_per_channel_max.max(dim=1)[0]


        lst_mse_loss_per_channel.append(mse_loss_per_channel.tolist())
        lst_mse_loss_per_channel_max.append(mse_loss_per_channel_max.tolist())
        lst_theoretical_eps_per_channel.append([eps for i in range(len(curr_unique_flipped_indices)) ])
        mse_loss_per_image = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_image = torch.sqrt(mse_loss_per_image.sum(dim=1))
        lst_mse_loss_per_image.append(mse_loss_per_image.tolist())
        #print(f"L2 per image : {mse_loss_per_image}")

        #print(f"L2 per channel : {mse_loss_per_channel}")

        """
        mse_loss_per_pixel = MSELoss_none(images_cw_base[0], images_benign[0])
        mse_loss_per_pixel = torch.sqrt(mse_loss_per_pixel.sum(dim=1)).flatten(1).mean(dim=1).mean()
        lst_mse_loss_per_pixel.append(mse_loss_per_pixel.item())
        #print(f"L2 per pixel : {mse_loss_per_pixel}")

        mse_max_per_pixel = MSELoss_none(images_cw_base[0], images_benign[0])
        mse_max_per_pixel = torch.sqrt(mse_max_per_pixel.sum(dim=1).flatten(1).max(dim=1)[0]).mean()
        lst_mse_max_per_pixel.append(mse_max_per_pixel.item())
        #print(f"L2 max pixel : {mse_max_per_pixel}")


        """

    lst_mse_loss_per_channel = [x for lst in lst_mse_loss_per_channel for x in lst]
    lst_theoretical_eps_per_channel = [x for lst in lst_theoretical_eps_per_channel for x in lst]
    lst_mse_loss_per_image = [x for lst in lst_mse_loss_per_image for x in lst]
    lst_mse_loss_per_channel_max = [x for lst in lst_mse_loss_per_channel_max for x in lst] 

    """all_counts
    lst_mse_loss_per_pixel = np.round(np.asarray(lst_mse_loss_per_pixel),decimals=4)
    lst_mse_max_per_pixel = np.round(np.asarray(lst_mse_max_per_pixel),decimals=4)

    """

    # Append to main lists
    all_mse_loss_per_channel.append(lst_mse_loss_per_channel.copy())
    all_theoretical_eps_per_channel.append(lst_theoretical_eps_per_channel.copy())
    all_mse_loss_per_image.append(lst_mse_loss_per_image.copy())
    all_mse_loss_per_channel_max.append(lst_mse_loss_per_channel_max.copy())
    """
    all_mse_loss_per_pixel.append(lst_mse_loss_per_pixel)
    all_mse_max_per_pixel.append(lst_mse_max_per_pixel)

    """
    



#all_mse_loss_per_channel = np.asarray(all_mse_loss_per_channel, dtype=object)
"""
all_mse_loss_per_pixel = np.asarray(all_mse_loss_per_pixel, dtype=object)
all_mse_max_per_pixel = np.asarray(all_mse_max_per_pixel, dtype=object)
all_mse_loss_per_image = np.asarray(all_mse_loss_per_image, dtype=object)
"""
    

color_list = sns.color_palette(n_colors=100)





# Theoretical Epsilon - Channelwise

In [ ]:
max_count = max(len(x) for x in all_theoretical_eps_per_channel)
max_eps_list = max(enumerate(all_theoretical_eps_per_channel), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_channel)):
    none_arr = [np.nan for k in range(max_count - len(all_theoretical_eps_per_channel[i]))]
    all_theoretical_eps_per_channel[i].extend(none_arr)


all_theoretical_eps_per_channel = np.round(all_theoretical_eps_per_channel, decimals=8)



# Create sample dataset
arch_rob = {
                 }
df = pd.DataFrame.from_dict(arch_rob)
sns.set(font_scale=0.7)

base_path = plot_path





In [ ]:


color_list = sns.color_palette(n_colors=100)

plot_config = "theoretical_epsilon_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config)


plot_config = "theoretical_epsilon_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07,xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config)

plot_config = "theoretical_epsilon_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )



xlim=0.05
# Plot CDFs


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Theoretical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "theoretical_epsilon_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')








# Empiricial Epsilon - Channelwise

In [ ]:

# Preprocess for Nones
max_count = max(len(x) for x in all_mse_loss_per_channel)
max_eps_list = max(enumerate(all_mse_loss_per_channel), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_channel)):
    none_arr = [np.nan for k in range(max_count - len(all_mse_loss_per_channel[i]))]
    all_mse_loss_per_channel[i].extend(none_arr)


all_mse_loss_per_channel = np.round(all_mse_loss_per_channel, decimals=8)



# Create sample dataset
arch_rob = {"ResNet" : all_mse_loss_per_channel[0], "MLP-3" : all_mse_loss_per_channel[1],  
            "MLP-6": all_mse_loss_per_channel[2],"ResNet_Rand_2x" : all_mse_loss_per_channel[3],
              "ResNet_Rand_4x" : all_mse_loss_per_channel[4], "MLP-12-Skip" : all_mse_loss_per_channel[5],
            "ResNet_Rand_Gaussian" : all_mse_loss_per_channel[6], "ResNet_SplitKernels" : all_mse_loss_per_channel[7],
             "ResNet_Rand_Gaussian_Data_0.015" : all_mse_loss_per_channel[8], "ResNet_Rand_Gaussian_Weights_0.015" : all_mse_loss_per_channel[9],
             "ResNet_Rand_2x_Cov" : all_mse_loss_per_channel[10],
              "ResNet_Unbounded": all_mse_loss_per_channel[11],
              "ResNet_Rand_Gaussian_Data_0.03" : all_mse_loss_per_channel[12],
             "ResNet_Rand_Gaussian_Data_0.045" : all_mse_loss_per_channel[13],
             "ResNet_Rand_Gaussian_Data_0.06" : all_mse_loss_per_channel[14],
              "ResNet_Rand_Gaussian_Weights_0.0075" : all_mse_loss_per_channel[15],
             "ResNet_Rand_Gaussian_Weights_0.0225" : all_mse_loss_per_channel[16],
 }
df = pd.DataFrame.from_dict(arch_rob)






In [ ]:

color_list = sns.color_palette(n_colors=100)

sns.set(font_scale=0.7)

base_path = plot_path

plot_config = "empirical_epsilon_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)


plot_config = "empirical_epsilon_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Empirical Epsilon - Per-channel", ylim=370,xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Empirical Epsilon - Per-channel", ylim=370,xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Empirical Epsilon - Per-channel", ylim=370,xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Empirical Epsilon - Per-channel", xlim=0.07, ylim=370, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)

plot_config = "empirical_epsilon_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_ResNet_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Unbounded.jpg"
plot_histplot(df=df, color=color_list[47], label='ResNet_Unbounded', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config , auto_bins=True)


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )






xlim=0.05

# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='MLP-3', color=color_list[1], label='MLP-3')
ax = sns.ecdfplot(data=df, x='MLP-6', color=color_list[2], label='MLP-6')
ax = sns.ecdfplot(data=df, x='MLP-12-Skip', color=color_list[3], label='MLP-12-Skip')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_channelwise_mlp_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x', color=color_list[4], label='ResNet_Rand_2x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_4x', color=color_list[5], label='ResNet_Rand_4x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x_Cov', color=color_list[99], label='ResNet_Rand_2x_Cov')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_channelwise_randCNN_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.015', color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.03', color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.045', color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.06', color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_channelwise_Gaussian_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')



# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')








# Empirical Epsilon - Imagewise

In [ ]:

# Preprocess for Nones
max_count = max(len(x) for x in all_mse_loss_per_image)
max_eps_list = max(enumerate(all_mse_loss_per_image), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_image)):
    none_arr = [np.nan for k in range(max_count - len(all_mse_loss_per_image[i]))]
    all_mse_loss_per_image[i].extend(none_arr)


all_mse_loss_per_image = np.round(all_mse_loss_per_image, decimals=8)



# Create sample dataset
arch_rob = {"ResNet" : all_mse_loss_per_image[0], "MLP-3" : all_mse_loss_per_image[1],  
            "MLP-6": all_mse_loss_per_image[2],"ResNet_Rand_2x" : all_mse_loss_per_image[3],
              "ResNet_Rand_4x" : all_mse_loss_per_image[4], "MLP-12-Skip" : all_mse_loss_per_image[5],
            "ResNet_Rand_Gaussian" : all_mse_loss_per_image[6], "ResNet_SplitKernels" : all_mse_loss_per_image[7],
             "ResNet_Rand_Gaussian_Data_0.015" : all_mse_loss_per_image[8], "ResNet_Rand_Gaussian_Weights_0.015" : all_mse_loss_per_image[9],
             "ResNet_Rand_2x_Cov" : all_mse_loss_per_image[10],
             "ResNet_Unbounded" : all_mse_loss_per_image[11],
              "ResNet_Rand_Gaussian_Data_0.03" : all_mse_loss_per_image[12],
             "ResNet_Rand_Gaussian_Data_0.045" : all_mse_loss_per_image[13],
             "ResNet_Rand_Gaussian_Data_0.06" : all_mse_loss_per_image[14],
            "ResNet_Rand_Gaussian_Weights_0.0075" : all_mse_loss_per_image[15],
             "ResNet_Rand_Gaussian_Weights_0.0225" : all_mse_loss_per_image[16],
               }
df = pd.DataFrame.from_dict(arch_rob)



In [ ]:

sns.set(font_scale=0.7)

base_path = plot_path

plot_config = "empirical_epsilon_imagewise_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)


plot_config = "empirical_epsilon_imagewise_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Empirical Epsilon - Per-Image", ylim=370,xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_imagewise_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Empirical Epsilon - Per-Image", ylim=370,xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_imagewise_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Empirical Epsilon - Per-Image", ylim=370,xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Empirical Epsilon - Per-Image", xlim=1.5, ylim=370, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Unbounded.jpg"
plot_histplot(df=df, color=color_list[47], label='ResNet_Unbounded', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )



plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True  )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True  )






xlim=1.9
# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')

ax = sns.ecdfplot(data=df, x='MLP-3', color=color_list[1], label='MLP-3')
ax = sns.ecdfplot(data=df, x='MLP-6', color=color_list[2], label='MLP-6')
ax = sns.ecdfplot(data=df, x='MLP-12-Skip', color=color_list[3], label='MLP-12-Skip')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_MLPs_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x', color=color_list[4], label='ResNet_Rand_2x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_4x', color=color_list[5], label='ResNet_Rand_4x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x_Cov', color=color_list[99], label='ResNet_Rand_2x_Cov')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_randCNN_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.015', color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.03', color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.045', color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.06', color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_Gaussian_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')





# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')



# Empircial Epsilon - Channelwise Max

In [ ]:

# Preprocess for Nones
max_count = max(len(x) for x in all_mse_loss_per_channel_max)
max_eps_list = max(enumerate(all_mse_loss_per_channel_max), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_channel_max)):
    none_arr = [np.nan for k in range(max_count - len(all_mse_loss_per_channel_max[i]))]
    all_mse_loss_per_channel_max[i].extend(none_arr)


all_mse_loss_per_channel_max = np.round(all_mse_loss_per_channel_max, decimals=8)


# Create sample dataset
arch_rob = {"ResNet" : all_mse_loss_per_channel_max[0], "MLP-3" : all_mse_loss_per_channel_max[1],  
            "MLP-6": all_mse_loss_per_channel_max[2],"ResNet_Rand_2x" : all_mse_loss_per_channel_max[3],
              "ResNet_Rand_4x" : all_mse_loss_per_channel_max[4], "MLP-12-Skip" : all_mse_loss_per_channel_max[5],
            "ResNet_Rand_Gaussian" : all_mse_loss_per_channel_max[6], "ResNet_SplitKernels" : all_mse_loss_per_channel_max[7],
             "ResNet_Rand_Gaussian_Data_0.015" : all_mse_loss_per_channel_max[8], "ResNet_Rand_Gaussian_Weights_0.015" : all_mse_loss_per_channel_max[9],
             "ResNet_Rand_2x_Cov" : all_mse_loss_per_channel_max[10],
              "ResNet_Unbounded": all_mse_loss_per_channel_max[11],
              "ResNet_Rand_Gaussian_Data_0.03" : all_mse_loss_per_channel_max[12],
             "ResNet_Rand_Gaussian_Data_0.045" : all_mse_loss_per_channel_max[13],
             "ResNet_Rand_Gaussian_Data_0.06" : all_mse_loss_per_channel_max[14],
            "ResNet_Rand_Gaussian_Weights_0.0075" : all_mse_loss_per_channel_max[15],
            "ResNet_Rand_Gaussian_Weights_0.0225" : all_mse_loss_per_channel_max[16],
           
              }
df = pd.DataFrame.from_dict(arch_rob)






In [ ]:

color_list = sns.color_palette(n_colors=100)

sns.set(font_scale=0.7)

base_path = plot_path

plot_config = "empirical_epsilon_max_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)


plot_config = "empirical_epsilon_max_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370,xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_max_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370,xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_max_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370,xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Empirical Epsilon - Per-channel-max", xlim=0.1, ylim=370, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)

plot_config = "empirical_epsilon_max_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Unbounded.jpg"
plot_histplot(df=df, color=color_list[47], label='ResNet_Unbounded', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )



plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Theoretical Epsilon - Per-channel", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,  auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Theoretical Epsilon - Per-channel", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,  auto_bins=True )





xlim=0.125
# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='MLP-3', color=color_list[1], label='MLP-3')
ax = sns.ecdfplot(data=df, x='MLP-6', color=color_list[2], label='MLP-6')
ax = sns.ecdfplot(data=df, x='MLP-12-Skip', color=color_list[3], label='MLP-12-Skip')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_channelwise_MLPs_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x', color=color_list[4], label='ResNet_Rand_2x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_4x', color=color_list[5], label='ResNet_Rand_4x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x_Cov', color=color_list[99], label='ResNet_Rand_2x_Cov')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_channelwise_randCNN_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.015', color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.03', color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.045', color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.06', color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_channelwise_Gaussian_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')






# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')







####################################################################################################################################################################################



# Old Attack with Pixel clippings

In [ ]:
all_base_paths = ["/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_f9c2570b43d17cd3f46b942f31d6fb03", 
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_a75d060c2415e722a953e03729658a14",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_604c7156b94d79a9430e2f87fd6d30ea",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_43528ed29912dd1a3825f30bea8e4ba8",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_814e4c261078e7110bf94f724cd2187e",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_2ee73fd3b0de2bb94001e8d73c2884f6",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_72d42cc289b41779183f931a5a35d003",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_1f77b0596b5392db3af2aff2a02444c8",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_cc21ecb149c2b45581f6079b6d36b9ea",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_970bf9406a461b0da3309170da242c10",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_9e525232280f7a4f7a11fb94d7e0a9d3",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_13cbebfb7cc94fe50dcb340f8a71799c",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_49016027b60adb7d413670ce48463514",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_05665f4b422e4c4bccf7cf81892232b8",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_b40c26bee285a2ec46710d0558c976fd",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_d2886fff253d02d0285e2b64b6a78e89",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_ce76e6e8f298f88256a6788b295ad0de",
                  #"/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_13cbebfb7cc94fe50dcb340f8a71799c"

                  ]

MSELoss_none = nn.MSELoss(reduction='none')
MSELoss = nn.MSELoss(reduction='mean')
flatten = nn.Flatten()
num_samples = 2560
og_num_samples = 2560
detector = 'Regular'
c_base=0.3
d_base=0.0
all_eps_list=[[0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04,  0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068],
           [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],           
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068, 0.07, 0.072, 0.074, 0.076, 0.078],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068, 0.07, 0.072, 0.074, 0.076, 0.078],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068, 0.07, 0.072, 0.074, 0.076, 0.078],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068, 0.07, 0.072, 0.074, 0.076, 0.078],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068, 0.07, 0.072, 0.074, 0.076, 0.078],
                        
            ]

plot_path = os.path.join(all_base_paths[0], "Plots")

# 0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018,

In [ ]:
plot_path

In [ ]:

all_freq_lists = []
all_theoretical_eps_per_channel = []
all_mse_loss_per_channel = []
all_mse_loss_per_pixel = []
all_mse_max_per_pixel = []
all_mse_loss_per_image = []
all_mse_loss_per_channel_max = []




for eps_idx, base_path in enumerate(all_base_paths):
    if "MT_Baseline_600547deae5b3d506d5c0befd9134c77" in base_path:
        num_samples = 1500
        c_base=1.0
    elif "MT_Baseline_604c7156b94d79a9430e2f87fd6d30ea" in base_path or "MT_Baseline_2ee73fd3b0de2bb94001e8d73c2884f6" in base_path or "MT_Baseline_9e525232280f7a4f7a11fb94d7e0a9d3" in base_path:
        num_samples = 3560
        c_base=1.0
    elif "MT_Baseline_43528ed29912dd1a3825f30bea8e4ba8" in base_path or "MT_Baseline_814e4c261078e7110bf94f724cd2187e"  in base_path:
        num_samples = 2500
        c_base=1.0
    elif "MT_Baseline_72d42cc289b41779183f931a5a35d003" in base_path or "MT_Baseline_1f77b0596b5392db3af2aff2a02444c8" in base_path or "MT_Baseline_cc21ecb149c2b45581f6079b6d36b9ea" in base_path or "MT_Baseline_b40c26bee285a2ec46710d0558c976fd" in base_path or "MT_Baseline_05665f4b422e4c4bccf7cf81892232b8" in base_path or "MT_Baseline_49016027b60adb7d413670ce48463514" in base_path:
        num_samples = 2560
        c_base=1.0
    elif "MT_Baseline_970bf9406a461b0da3309170da242c10" in base_path or "MT_Baseline_d2886fff253d02d0285e2b64b6a78e89" in base_path or "MT_Baseline_ce76e6e8f298f88256a6788b295ad0de" in base_path:
        num_samples = 4560
        c_base=1.0
    elif "MT_Baseline_13cbebfb7cc94fe50dcb340f8a71799c" in base_path:
        num_samples = 3560
        c_base=1.0
    else:
        num_samples = og_num_samples
        c_base=0.3
    all_flipped_indices = []
    lst_theoretical_eps_per_channel = []
    lst_mse_loss_per_channel = []
    lst_mse_loss_per_pixel = []
    lst_mse_max_per_pixel = []
    lst_mse_loss_per_image = []
    lst_mse_loss_per_channel_max = []

    prev_flipped_indices = set()
    for eps in (all_eps_list[eps_idx]):
        images_cw_base = f"Adversarial_Datasets/CW_adv_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        images_benign = f"Benign_Datasets/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        predictions_base = f"Predictions/Model/CW_type-adversarial_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        filtered_indices_base = f"Predictions/Perturbed_Samples/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pt"
        images_cw_base = os.path.join(base_path, images_cw_base)
        images_benign = os.path.join(base_path, images_benign)
        predictions_base_path = os.path.join(base_path, predictions_base)
        filtered_indices_base_path = os.path.join(base_path, filtered_indices_base)

        # Filter by common indices
        filtered_indices_base = torch.load(filtered_indices_base_path)
        filtered_indices_base = [x.item() for x in filtered_indices_base]
        curr_flipped_indices = set(filtered_indices_base)
        curr_unique_flipped_indices = curr_flipped_indices.difference(prev_flipped_indices)    
        prev_flipped_indices.update(curr_unique_flipped_indices)
        curr_unique_flipped_indices = list(curr_unique_flipped_indices)
        all_flipped_indices.append(filtered_indices_base)

        with open(images_cw_base, 'rb') as in_file:
            images_cw_base = pickle.load(in_file)
            images_cw_base[0] = images_cw_base[0][curr_unique_flipped_indices]
            images_cw_base[1] = images_cw_base[1][curr_unique_flipped_indices]
            
        with open(images_benign, 'rb') as in_file:
            images_benign = pickle.load(in_file)
            images_benign[0] = images_benign[0][curr_unique_flipped_indices]
            images_benign[1] = images_benign[1][curr_unique_flipped_indices]

        with open(predictions_base_path, 'rb') as in_file:
            predictions = pickle.load(in_file)
            true_labels_base = np.asarray(predictions['true_labels'])[curr_unique_flipped_indices]
            pred_labels_base = np.asarray(predictions['pred_labels'])[curr_unique_flipped_indices]

        mse_loss_per_channel = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_channel = torch.sqrt(mse_loss_per_channel)
        mse_loss_per_channel = mse_loss_per_channel.mean(dim=1)

        mse_loss_per_channel_max = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_channel_max = torch.sqrt(mse_loss_per_channel_max)
        mse_loss_per_channel_max = mse_loss_per_channel_max.max(dim=1)[0]


        lst_mse_loss_per_channel.append(mse_loss_per_channel.tolist())
        lst_mse_loss_per_channel_max.append(mse_loss_per_channel_max.tolist())
        lst_theoretical_eps_per_channel.append([eps for i in range(len(curr_unique_flipped_indices)) ])
        mse_loss_per_image = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_image = torch.sqrt(mse_loss_per_image.sum(dim=1))
        lst_mse_loss_per_image.append(mse_loss_per_image.tolist())
        #print(f"L2 per image : {mse_loss_per_image}")

        #print(f"L2 per channel : {mse_loss_per_channel}")

        """
        mse_loss_per_pixel = MSELoss_none(images_cw_base[0], images_benign[0])
        mse_loss_per_pixel = torch.sqrt(mse_loss_per_pixel.sum(dim=1)).flatten(1).mean(dim=1).mean()
        lst_mse_loss_per_pixel.append(mse_loss_per_pixel.item())
        #print(f"L2 per pixel : {mse_loss_per_pixel}")

        mse_max_per_pixel = MSELoss_none(images_cw_base[0], images_benign[0])
        mse_max_per_pixel = torch.sqrt(mse_max_per_pixel.sum(dim=1).flatten(1).max(dim=1)[0]).mean()
        lst_mse_max_per_pixel.append(mse_max_per_pixel.item())
        #print(f"L2 max pixel : {mse_max_per_pixel}")


        """

    lst_mse_loss_per_channel = [x for lst in lst_mse_loss_per_channel for x in lst]
    lst_theoretical_eps_per_channel = [x for lst in lst_theoretical_eps_per_channel for x in lst]
    lst_mse_loss_per_image = [x for lst in lst_mse_loss_per_image for x in lst]
    lst_mse_loss_per_channel_max = [x for lst in lst_mse_loss_per_channel_max for x in lst] 

    """all_counts
    lst_mse_loss_per_pixel = np.round(np.asarray(lst_mse_loss_per_pixel),decimals=4)
    lst_mse_max_per_pixel = np.round(np.asarray(lst_mse_max_per_pixel),decimals=4)

    """

    # Append to main lists
    all_mse_loss_per_channel.append(lst_mse_loss_per_channel.copy())
    all_theoretical_eps_per_channel.append(lst_theoretical_eps_per_channel.copy())
    all_mse_loss_per_image.append(lst_mse_loss_per_image.copy())
    all_mse_loss_per_channel_max.append(lst_mse_loss_per_channel_max.copy())
    """
    all_mse_loss_per_pixel.append(lst_mse_loss_per_pixel)
    all_mse_max_per_pixel.append(lst_mse_max_per_pixel)

    """
    



#all_mse_loss_per_channel = np.asarray(all_mse_loss_per_channel, dtype=object)
"""
all_mse_loss_per_pixel = np.asarray(all_mse_loss_per_pixel, dtype=object)
all_mse_max_per_pixel = np.asarray(all_mse_max_per_pixel, dtype=object)
all_mse_loss_per_image = np.asarray(all_mse_loss_per_image, dtype=object)
"""
    

color_list = sns.color_palette(n_colors=100)





# Theoretical Epsilon - Channelwise

In [ ]:
max_count = max(len(x) for x in all_theoretical_eps_per_channel)
max_eps_list = max(enumerate(all_theoretical_eps_per_channel), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_channel)):
    none_arr = [np.nan for k in range(max_count - len(all_theoretical_eps_per_channel[i]))]
    all_theoretical_eps_per_channel[i].extend(none_arr)


all_theoretical_eps_per_channel = np.round(all_theoretical_eps_per_channel, decimals=8)



# Create sample dataset
arch_rob = {"ResNet" : all_theoretical_eps_per_channel[0], "MLP-3" : all_theoretical_eps_per_channel[1],  
            "MLP-6": all_theoretical_eps_per_channel[2],"ResNet_Rand_2x" : all_theoretical_eps_per_channel[3], 
            "ResNet_Rand_4x" : all_theoretical_eps_per_channel[4],"MLP-12-Skip" : all_theoretical_eps_per_channel[5],
             "ResNet_Rand_Gaussian" : all_theoretical_eps_per_channel[6], "ResNet_SplitKernels" : all_theoretical_eps_per_channel[7],
             "ResNet_Rand_Gaussian_Data_0.015" : all_theoretical_eps_per_channel[8], "ResNet_Rand_Gaussian_Weights_0.015" : all_theoretical_eps_per_channel[9],
             "ResNet_Rand_2x_Cov" : all_theoretical_eps_per_channel[10],
             "ResNet_Rand_Gaussian_Data_0.03" : all_theoretical_eps_per_channel[12],
             "ResNet_Rand_Gaussian_Data_0.045" : all_theoretical_eps_per_channel[13],
             "ResNet_Rand_Gaussian_Data_0.06" : all_theoretical_eps_per_channel[14],
             "ResNet_Rand_Gaussian_Weights_0.0075" : all_theoretical_eps_per_channel[15],
             "ResNet_Rand_Gaussian_Weights_0.0225" : all_theoretical_eps_per_channel[16],



                 }
df = pd.DataFrame.from_dict(arch_rob)
sns.set(font_scale=0.7)

base_path = plot_path





In [ ]:


color_list = sns.color_palette(n_colors=100)

plot_config = "theoretical_epsilon_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config)


plot_config = "theoretical_epsilon_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07,xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config)

plot_config = "theoretical_epsilon_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Theoretical Epsilon - Per-channel", ylim=300, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )

plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )


plot_config = "theoretical_epsilon_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Theoretical Epsilon - Per-channel", ylim=600, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config )



xlim=0.05
# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')

ax = sns.ecdfplot(data=df, x='MLP-3', color=color_list[1], label='MLP-3')
ax = sns.ecdfplot(data=df, x='MLP-6', color=color_list[2], label='MLP-6')
ax = sns.ecdfplot(data=df, x='MLP-12-Skip', color=color_list[3], label='MLP-12-Skip')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Theoretical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "theoretical_epsilon_mlps_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x', color=color_list[4], label='ResNet_Rand_2x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_4x', color=color_list[5], label='ResNet_Rand_4x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x_Cov', color=color_list[99], label='ResNet_Rand_2x_Cov')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Theoretical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "theoretical_epsilon_randCNN_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.015', color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.03', color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.045', color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.06', color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Theoretical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "theoretical_epsilon_Gaussians_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')





# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Theoretical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "theoretical_epsilon_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')








# Empiricial Epsilon - Channelwise

In [ ]:

# Preprocess for Nones
max_count = max(len(x) for x in all_mse_loss_per_channel)
max_eps_list = max(enumerate(all_mse_loss_per_channel), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_channel)):
    none_arr = [np.nan for k in range(max_count - len(all_mse_loss_per_channel[i]))]
    all_mse_loss_per_channel[i].extend(none_arr)


all_mse_loss_per_channel = np.round(all_mse_loss_per_channel, decimals=8)



# Create sample dataset
arch_rob = {"ResNet" : all_mse_loss_per_channel[0], "MLP-3" : all_mse_loss_per_channel[1],  
            "MLP-6": all_mse_loss_per_channel[2],"ResNet_Rand_2x" : all_mse_loss_per_channel[3],
              "ResNet_Rand_4x" : all_mse_loss_per_channel[4], "MLP-12-Skip" : all_mse_loss_per_channel[5],
            "ResNet_Rand_Gaussian" : all_mse_loss_per_channel[6], "ResNet_SplitKernels" : all_mse_loss_per_channel[7],
             "ResNet_Rand_Gaussian_Data_0.015" : all_mse_loss_per_channel[8], "ResNet_Rand_Gaussian_Weights_0.015" : all_mse_loss_per_channel[9],
             "ResNet_Rand_2x_Cov" : all_mse_loss_per_channel[10],
              "ResNet_Unbounded": all_mse_loss_per_channel[11],
              "ResNet_Rand_Gaussian_Data_0.03" : all_mse_loss_per_channel[12],
             "ResNet_Rand_Gaussian_Data_0.045" : all_mse_loss_per_channel[13],
             "ResNet_Rand_Gaussian_Data_0.06" : all_mse_loss_per_channel[14],
              "ResNet_Rand_Gaussian_Weights_0.0075" : all_mse_loss_per_channel[15],
             "ResNet_Rand_Gaussian_Weights_0.0225" : all_mse_loss_per_channel[16],
 }
df = pd.DataFrame.from_dict(arch_rob)






In [ ]:

color_list = sns.color_palette(n_colors=100)

sns.set(font_scale=0.7)

base_path = plot_path

plot_config = "empirical_epsilon_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)


plot_config = "empirical_epsilon_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Empirical Epsilon - Per-channel", ylim=370,xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Empirical Epsilon - Per-channel", ylim=370,xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Empirical Epsilon - Per-channel", ylim=370,xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Empirical Epsilon - Per-channel", xlim=0.07, ylim=370, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)

plot_config = "empirical_epsilon_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_ResNet_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Unbounded.jpg"
plot_histplot(df=df, color=color_list[47], label='ResNet_Unbounded', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config , auto_bins=True)


plot_config = "empirical_epsilon_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=0.07, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )






xlim=0.05

# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='MLP-3', color=color_list[1], label='MLP-3')
ax = sns.ecdfplot(data=df, x='MLP-6', color=color_list[2], label='MLP-6')
ax = sns.ecdfplot(data=df, x='MLP-12-Skip', color=color_list[3], label='MLP-12-Skip')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_channelwise_mlp_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x', color=color_list[4], label='ResNet_Rand_2x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_4x', color=color_list[5], label='ResNet_Rand_4x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x_Cov', color=color_list[99], label='ResNet_Rand_2x_Cov')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_channelwise_randCNN_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.015', color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.03', color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.045', color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.06', color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_channelwise_Gaussian_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')



# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')








# Empirical Epsilon - Imagewise

In [ ]:

# Preprocess for Nones
max_count = max(len(x) for x in all_mse_loss_per_image)
max_eps_list = max(enumerate(all_mse_loss_per_image), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_image)):
    none_arr = [np.nan for k in range(max_count - len(all_mse_loss_per_image[i]))]
    all_mse_loss_per_image[i].extend(none_arr)


all_mse_loss_per_image = np.round(all_mse_loss_per_image, decimals=8)



# Create sample dataset
arch_rob = {"ResNet" : all_mse_loss_per_image[0], "MLP-3" : all_mse_loss_per_image[1],  
            "MLP-6": all_mse_loss_per_image[2],"ResNet_Rand_2x" : all_mse_loss_per_image[3],
              "ResNet_Rand_4x" : all_mse_loss_per_image[4], "MLP-12-Skip" : all_mse_loss_per_image[5],
            "ResNet_Rand_Gaussian" : all_mse_loss_per_image[6], "ResNet_SplitKernels" : all_mse_loss_per_image[7],
             "ResNet_Rand_Gaussian_Data_0.015" : all_mse_loss_per_image[8], "ResNet_Rand_Gaussian_Weights_0.015" : all_mse_loss_per_image[9],
             "ResNet_Rand_2x_Cov" : all_mse_loss_per_image[10],
             "ResNet_Unbounded" : all_mse_loss_per_image[11],
              "ResNet_Rand_Gaussian_Data_0.03" : all_mse_loss_per_image[12],
             "ResNet_Rand_Gaussian_Data_0.045" : all_mse_loss_per_image[13],
             "ResNet_Rand_Gaussian_Data_0.06" : all_mse_loss_per_image[14],
            "ResNet_Rand_Gaussian_Weights_0.0075" : all_mse_loss_per_image[15],
             "ResNet_Rand_Gaussian_Weights_0.0225" : all_mse_loss_per_image[16],
               }
df = pd.DataFrame.from_dict(arch_rob)



In [ ]:

sns.set(font_scale=0.7)

base_path = plot_path

plot_config = "empirical_epsilon_imagewise_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)


plot_config = "empirical_epsilon_imagewise_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Empirical Epsilon - Per-Image", ylim=370,xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_imagewise_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Empirical Epsilon - Per-Image", ylim=370,xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_imagewise_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Empirical Epsilon - Per-Image", ylim=370,xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Empirical Epsilon - Per-Image", xlim=1.5, ylim=370, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Unbounded.jpg"
plot_histplot(df=df, color=color_list[47], label='ResNet_Unbounded', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )



plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Empirical Epsilon - Per-Image", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True  )


plot_config = "empirical_epsilon_imagewise_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Empirical Epsilon - Per-channel", ylim=370, xlim=1.5, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True  )






xlim=1.9
# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')

ax = sns.ecdfplot(data=df, x='MLP-3', color=color_list[1], label='MLP-3')
ax = sns.ecdfplot(data=df, x='MLP-6', color=color_list[2], label='MLP-6')
ax = sns.ecdfplot(data=df, x='MLP-12-Skip', color=color_list[3], label='MLP-12-Skip')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_MLPs_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x', color=color_list[4], label='ResNet_Rand_2x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_4x', color=color_list[5], label='ResNet_Rand_4x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x_Cov', color=color_list[99], label='ResNet_Rand_2x_Cov')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_randCNN_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.015', color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.03', color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.045', color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.06', color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_Gaussian_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')





# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Imagewise")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_imagewise_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')



# Empircial Epsilon - Channelwise Max

In [ ]:

# Preprocess for Nones
max_count = max(len(x) for x in all_mse_loss_per_channel_max)
max_eps_list = max(enumerate(all_mse_loss_per_channel_max), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_channel_max)):
    none_arr = [np.nan for k in range(max_count - len(all_mse_loss_per_channel_max[i]))]
    all_mse_loss_per_channel_max[i].extend(none_arr)


all_mse_loss_per_channel_max = np.round(all_mse_loss_per_channel_max, decimals=8)


# Create sample dataset
arch_rob = {"ResNet" : all_mse_loss_per_channel_max[0], "MLP-3" : all_mse_loss_per_channel_max[1],  
            "MLP-6": all_mse_loss_per_channel_max[2],"ResNet_Rand_2x" : all_mse_loss_per_channel_max[3],
              "ResNet_Rand_4x" : all_mse_loss_per_channel_max[4], "MLP-12-Skip" : all_mse_loss_per_channel_max[5],
            "ResNet_Rand_Gaussian" : all_mse_loss_per_channel_max[6], "ResNet_SplitKernels" : all_mse_loss_per_channel_max[7],
             "ResNet_Rand_Gaussian_Data_0.015" : all_mse_loss_per_channel_max[8], "ResNet_Rand_Gaussian_Weights_0.015" : all_mse_loss_per_channel_max[9],
             "ResNet_Rand_2x_Cov" : all_mse_loss_per_channel_max[10],
              "ResNet_Unbounded": all_mse_loss_per_channel_max[11],
              "ResNet_Rand_Gaussian_Data_0.03" : all_mse_loss_per_channel_max[12],
             "ResNet_Rand_Gaussian_Data_0.045" : all_mse_loss_per_channel_max[13],
             "ResNet_Rand_Gaussian_Data_0.06" : all_mse_loss_per_channel_max[14],
            "ResNet_Rand_Gaussian_Weights_0.0075" : all_mse_loss_per_channel_max[15],
            "ResNet_Rand_Gaussian_Weights_0.0225" : all_mse_loss_per_channel_max[16],
           
              }
df = pd.DataFrame.from_dict(arch_rob)






In [ ]:

color_list = sns.color_palette(n_colors=100)

sns.set(font_scale=0.7)

base_path = plot_path

plot_config = "empirical_epsilon_max_ResNet.jpg"
plot_histplot(df=df, color=color_list[0], label='ResNet', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)


plot_config = "empirical_epsilon_max_MLP-3.jpg"
plot_histplot(df=df, color=color_list[1], label='MLP-3', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370,xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_max_MLP-6.jpg"
plot_histplot(df=df, color=color_list[2], label='MLP-6', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370,xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_max_MLP-12.jpg"
plot_histplot(df=df, color=color_list[3], label='MLP-12-Skip', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370,xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_2x.jpg"
plot_histplot(df=df, color=color_list[4], label='ResNet_Rand_2x', title="PDF - Empirical Epsilon - Per-channel-max", xlim=0.1, ylim=370, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True)

plot_config = "empirical_epsilon_max_ResNet_Rand_4x.jpg"
plot_histplot(df=df, color=color_list[5], label='ResNet_Rand_4x', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1,  xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian.jpg"
plot_histplot(df=df, color=color_list[6], label='ResNet_Rand_Gaussian', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_ResNet_SplitKernels.jpg"
plot_histplot(df=df, color=color_list[7], label='ResNet_SplitKernels', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.015.jpg"
plot_histplot(df=df, color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Weights_0.015.jpg"
plot_histplot(df=df, color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_ResNet_Rand_2x_Cov.jpg"
plot_histplot(df=df, color=color_list[99], label='ResNet_Rand_2x_Cov', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Unbounded.jpg"
plot_histplot(df=df, color=color_list[47], label='ResNet_Unbounded', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.03.jpg"
plot_histplot(df=df, color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.045.jpg"
plot_histplot(df=df, color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )

plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Data_0.06.jpg"
plot_histplot(df=df, color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06', title="PDF - Empirical Epsilon - Per-channel-max", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config, auto_bins=True )



plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Weights_0.0075.jpg"
plot_histplot(df=df, color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075', title="PDF - Theoretical Epsilon - Per-channel", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,  auto_bins=True )


plot_config = "empirical_epsilon_max_ResNet_Rand_Gaussian_Weights_0.0225.jpg"
plot_histplot(df=df, color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225', title="PDF - Theoretical Epsilon - Per-channel", ylim=370, xlim=0.1, xlabel="Epsilon", ylabel="Flip count", base_path=base_path, plot_config=plot_config,  auto_bins=True )





xlim=0.125
# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='MLP-3', color=color_list[1], label='MLP-3')
ax = sns.ecdfplot(data=df, x='MLP-6', color=color_list[2], label='MLP-6')
ax = sns.ecdfplot(data=df, x='MLP-12-Skip', color=color_list[3], label='MLP-12-Skip')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_channelwise_MLPs_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x', color=color_list[4], label='ResNet_Rand_2x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_4x', color=color_list[5], label='ResNet_Rand_4x')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_2x_Cov', color=color_list[99], label='ResNet_Rand_2x_Cov')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_channelwise_randCNN_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')

plt.clf()


# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.015', color=color_list[8], label='ResNet_Rand_Gaussian_Data_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.03', color=color_list[11], label='ResNet_Rand_Gaussian_Data_0.03')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.045', color=color_list[12], label='ResNet_Rand_Gaussian_Data_0.045')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Data_0.06', color=color_list[13], label='ResNet_Rand_Gaussian_Data_0.06')
#ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_T', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_T')
ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_channelwise_Gaussian_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')






# Plot CDFs
plt.figure(figsize=(15, 15))
ax = sns.ecdfplot(data=df, x='ResNet', color=color_list[0], label='ResNet')
ax = sns.ecdfplot(data=df, x='ResNet_Unbounded', color=color_list[47], label='ResNet_Unbounded')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.015', color=color_list[34], label='ResNet_Rand_Gaussian_Weights_0.015')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0075', color=color_list[35], label='ResNet_Rand_Gaussian_Weights_0.0075')
ax = sns.ecdfplot(data=df, x='ResNet_Rand_Gaussian_Weights_0.0225', color=color_list[36], label='ResNet_Rand_Gaussian_Weights_0.0225')

ax.set(ylabel='Density')
ax.set(xlabel='Epsilon')
plt.title("CDF - Empirical Epsilon - Per-channel-max")
ax.legend(fontsize=15)
ax.set_xlim(0, xlim)
plot_config = "empirical_epsilon_max_Gaussians_Weights_cdf.jpg"
out_path = os.path.join(plot_path, plot_config)
plt.savefig(out_path, bbox_inches='tight')







####################################################################################################################################################################################



# Visualizations

# MASK VISUALIZATIONS

In [ ]:
def plot_side_by_side(image1, image2, image3, image4, idx, save_path, titles=None):
    """
    Plots three images side by side.

    Args:
        image1 (np.ndarray): The first image.
        image2 (np.ndarray): The second image.
        image3 (np.ndarray): The third image.
        titles (list, optional): A list of titles for the images.

    Returns:
        None
    """

    fig, axes = plt.subplots(1, 4, figsize=(12, 5))
    
    axes[0].set_title(titles[0])
    axes[0].imshow(image1)
    axes[0].axis('off')
    axes[1].set_title(titles[1])
    axes[1].imshow(image2)
    axes[1].axis('off')

    axes[2].set_title(titles[2])
    axes[2].imshow(image3)
    axes[2].axis('off')
    axes[3].set_title(titles[3])
    axes[3].imshow(image4)
    axes[3].axis('off')

    plt.tight_layout()
    plt.savefig("/home/zsarwar/Projects/SparseDNNs/outputs.jpg")

In [ ]:
all_base_paths = ["/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f_arch/MT_Baseline_f9c2570b43d17cd3f46b942f31d6fb03", 
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_13cbebfb7cc94fe50dcb340f8a71799c",
                  "/net/scratch/zsarwar/SparseDNNs/MT_cifar10_randCNN_10_8d64c1903fcb2ff6173fe22979c3175f/MT_Baseline_cc21ecb149c2b45581f6079b6d36b9ea"
                  ]

MSELoss_none = nn.MSELoss(reduction='none')
MSELoss = nn.MSELoss(reduction='mean')
flatten = nn.Flatten()
num_samples = 2560
og_num_samples = 2560
detector = 'Regular'
c_base=0.3
d_base=0.0
"""
all_eps_list=[
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.0],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04],
            [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03, 0.032, 0.034, 0.036, 0.038, 0.04, 0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.054, 0.056, 0.058, 0.060, 0.062, 0.064, 0.066, 0.068],
                
            ]
"""
all_eps_list=[
            [0.04],
            [0.0],
            [0.04],
            [0.04],
            ]

plot_path = os.path.join(all_base_paths[0], "Plots")

all_freq_lists = []
all_theoretical_eps_per_channel = []
all_mse_loss_per_channel = []
all_mse_loss_per_pixel = []
all_mse_max_per_pixel = []
all_mse_loss_per_image = []
all_mse_loss_per_channel_max = []
all_images_base = []
all_images_benign = []



for eps_idx, base_path in enumerate(all_base_paths):

    if "MT_Baseline_814e4c261078e7110bf94f724cd2187e"  in base_path:
        num_samples = 2500
        c_base=1.0
    elif "MT_Baseline_cc21ecb149c2b45581f6079b6d36b9ea" in base_path:
        num_samples = 2560
        c_base=1.0
    elif "MT_Baseline_13cbebfb7cc94fe50dcb340f8a71799c" in base_path:
        num_samples = 3560
        c_base=1.2
    else:
        num_samples = og_num_samples
        c_base=0.3
    all_flipped_indices = []
    lst_theoretical_eps_per_channel = []
    lst_mse_loss_per_channel = []
    lst_mse_loss_per_pixel = []
    lst_mse_max_per_pixel = []
    lst_mse_loss_per_image = []
    lst_mse_loss_per_channel_max = []

    prev_flipped_indices = set()
    for eps in (all_eps_list[eps_idx]):
        images_cw_base = f"Adversarial_Datasets/CW_adv_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        images_benign = f"Benign_Datasets/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        predictions_base = f"Predictions/Model/CW_type-adversarial_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
        filtered_indices_base = f"Predictions/Perturbed_Samples/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pt"
        images_cw_base = os.path.join(base_path, images_cw_base)
        images_benign = os.path.join(base_path, images_benign)
        predictions_base_path = os.path.join(base_path, predictions_base)
        filtered_indices_base_path = os.path.join(base_path, filtered_indices_base)

        # Filter by common indices
        filtered_indices_base = torch.load(filtered_indices_base_path)
        filtered_indices_base = [x.item() for x in filtered_indices_base]
        curr_flipped_indices = set(filtered_indices_base)
        curr_unique_flipped_indices = curr_flipped_indices.difference(prev_flipped_indices)    
        prev_flipped_indices.update(curr_unique_flipped_indices)
        curr_unique_flipped_indices = list(curr_unique_flipped_indices)
        all_flipped_indices.append(filtered_indices_base)

        with open(images_cw_base, 'rb') as in_file:
            images_cw_base = pickle.load(in_file)
            images_cw_base[0] = images_cw_base[0][curr_unique_flipped_indices]
            images_cw_base[1] = images_cw_base[1][curr_unique_flipped_indices]
            all_images_base.append(images_cw_base[0])

        with open(images_benign, 'rb') as in_file:
            images_benign = pickle.load(in_file)
            images_benign[0] = images_benign[0][curr_unique_flipped_indices]
            images_benign[1] = images_benign[1][curr_unique_flipped_indices]
            all_images_benign.append(images_benign[0])

        with open(predictions_base_path, 'rb') as in_file:
            predictions = pickle.load(in_file)
            true_labels_base = np.asarray(predictions['true_labels'])[curr_unique_flipped_indices]
            pred_labels_base = np.asarray(predictions['pred_labels'])[curr_unique_flipped_indices]

        mse_loss_per_channel = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_channel = torch.sqrt(mse_loss_per_channel)
        mse_loss_per_channel = mse_loss_per_channel.mean(dim=1)

        mse_loss_per_channel_max = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_channel_max = torch.sqrt(mse_loss_per_channel_max)
        mse_loss_per_channel_max = mse_loss_per_channel_max.max(dim=1)[0]


        lst_mse_loss_per_channel.append(mse_loss_per_channel.tolist())
        lst_mse_loss_per_channel_max.append(mse_loss_per_channel_max.tolist())
        lst_theoretical_eps_per_channel.append([eps for i in range(len(curr_unique_flipped_indices)) ])
        mse_loss_per_image = MSELoss_none(flatten(images_cw_base[0]), flatten(images_benign[0]))
        mse_loss_per_image = torch.sqrt(mse_loss_per_image.sum(dim=1))
        lst_mse_loss_per_image.append(mse_loss_per_image.tolist())
        #print(f"L2 per image : {mse_loss_per_image}")

        #print(f"L2 per channel : {mse_loss_per_channel}")

        """
        mse_loss_per_pixel = MSELoss_none(images_cw_base[0], images_benign[0])
        mse_loss_per_pixel = torch.sqrt(mse_loss_per_pixel.sum(dim=1)).flatten(1).mean(dim=1).mean()
        lst_mse_loss_per_pixel.append(mse_loss_per_pixel.item())
        #print(f"L2 per pixel : {mse_loss_per_pixel}")

        mse_max_per_pixel = MSELoss_none(images_cw_base[0], images_benign[0])
        mse_max_per_pixel = torch.sqrt(mse_max_per_pixel.sum(dim=1).flatten(1).max(dim=1)[0]).mean()
        lst_mse_max_per_pixel.append(mse_max_per_pixel.item())
        #print(f"L2 max pixel : {mse_max_per_pixel}")


        """

    lst_mse_loss_per_channel = [x for lst in lst_mse_loss_per_channel for x in lst]
    lst_theoretical_eps_per_channel = [x for lst in lst_theoretical_eps_per_channel for x in lst]
    lst_mse_loss_per_image = [x for lst in lst_mse_loss_per_image for x in lst]
    lst_mse_loss_per_channel_max = [x for lst in lst_mse_loss_per_channel_max for x in lst] 

    """all_counts
    lst_mse_loss_per_pixel = np.round(np.asarray(lst_mse_loss_per_pixel),decimals=4)
    lst_mse_max_per_pixel = np.round(np.asarray(lst_mse_max_per_pixel),decimals=4)

    """

    # Append to main lists
    all_mse_loss_per_channel.append(lst_mse_loss_per_channel.copy())
    all_theoretical_eps_per_channel.append(lst_theoretical_eps_per_channel.copy())
    all_mse_loss_per_image.append(lst_mse_loss_per_image.copy())
    all_mse_loss_per_channel_max.append(lst_mse_loss_per_channel_max.copy())
    """
    all_mse_loss_per_pixel.append(lst_mse_loss_per_pixel)
    all_mse_max_per_pixel.append(lst_mse_max_per_pixel)

    """
    



#all_mse_loss_per_channel = np.asarray(all_mse_loss_per_channel, dtype=object)
"""
all_mse_loss_per_pixel = np.asarray(all_mse_loss_per_pixel, dtype=object)
all_mse_max_per_pixel = np.asarray(all_mse_max_per_pixel, dtype=object)
all_mse_loss_per_image = np.asarray(all_mse_loss_per_image, dtype=object)
"""
    

color_list = sns.color_palette(n_colors=100)





In [ ]:
#ben_idxs = []
#resnet = []
#resnet_unbounded = []
#resnet_gaussian = []
for idx_0 in range(20):
    benign_0 = all_images_benign[0][idx_0]
    for idx_1 in range(len(all_images_benign[1])):
        benign_1 = all_images_benign[1][idx_1]
        if torch.all(benign_0 == benign_1): 
            found_match = False           
            for idx_2 in range(len(all_images_benign[2])):
                benign_2 = all_images_benign[2][idx_2]
                if torch.all(benign_1 == benign_2):
                    title_benign = f'GT'
                    title_unbounded = f'CW_Unbounded'
                    title_bounded = f'CW_Bounded'
                    title_gaussian_bounded = f'CW_Gaussian_Bounded'
                    titles = [title_benign, title_unbounded, title_bounded, title_gaussian_bounded]
                    benign_0 = benign_0.swapaxes(0,1)
                    benign_0 = benign_0.swapaxes(1,2)
                    
                    cw_bounded = all_images_base[0][idx_0]
                    cw_bounded = cw_bounded.swapaxes(0,1)
                    cw_bounded = cw_bounded.swapaxes(1,2)

                    cw_unbounded = all_images_base[1][idx_1]
                    cw_unbounded = cw_unbounded.swapaxes(0,1)
                    cw_unbounded = cw_unbounded.swapaxes(1,2)

                    cw_gaussian_bounded = all_images_base[2][idx_2]
                    cw_gaussian_bounded = cw_gaussian_bounded.swapaxes(0,1)
                    cw_gaussian_bounded = cw_gaussian_bounded.swapaxes(1,2)
                    
                    plot_side_by_side(benign_0, cw_unbounded, cw_bounded, cw_gaussian_bounded , idx_0, None, titles=titles)
                    found_match = True
                    break
            if found_match:
                break
                


In [ ]:
torch.all(benign_1 == benign_0)

In [ ]:

# Preprocess for Nones
max_count = max(len(x) for x in all_mse_loss_per_channel)
max_eps_list = max(enumerate(all_mse_loss_per_channel), key=lambda m : len(m[1]) )[0]

for i in range(len(all_mse_loss_per_channel)):
    none_arr = [np.nan for k in range(max_count - len(all_mse_loss_per_channel[i]))]
    all_mse_loss_per_channel[i].extend(none_arr)


all_mse_loss_per_channel = np.round(all_mse_loss_per_channel, decimals=8)



# Create sample dataset
arch_rob = {"ResNet" : all_mse_loss_per_channel[0],  
            "ResNet_Unbounded": all_mse_loss_per_channel[1],
            #"ResNet_Rand_4x" : all_mse_loss_per_channel[2], 
            "ResNet_Rand_Gaussian_Data_0.015" : all_mse_loss_per_channel[3],
}
df = pd.DataFrame.from_dict(arch_rob)






In [ ]:
df

In [ ]:
og_num_samples = 2500
all_freq_lists = []
all_theoretical_eps_per_channel = []
all_mse_loss_per_channel = []
all_mse_loss_per_pixel = []
all_mse_max_per_pixel = []
all_mse_loss_per_image = []
    
for eps_idx, base_path in enumerate(all_base_paths[3:4]):
    num_samples = 2500
    all_flipped_indices = []
    lst_theoretical_eps_per_channel = []
    lst_mse_loss_per_channel = []
    lst_mse_loss_per_pixel = []
    lst_mse_max_per_pixel = []
    lst_mse_loss_per_image = []

    prev_flipped_indices = set()
    eps = all_eps_list[3][-1]
    images_cw_base = f"Adversarial_Datasets/CW_adv_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    images_benign = f"Benign_Datasets/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    predictions_base = f"Predictions/Model/CW_type-adversarial_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    filtered_indices_base = f"Predictions/Perturbed_Samples/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pt"
    images_cw_base = os.path.join(base_path, images_cw_base)
    images_benign = os.path.join(base_path, images_benign)
    predictions_base_path = os.path.join(base_path, predictions_base)
    filtered_indices_base_path = os.path.join(base_path, filtered_indices_base)

    # Filter by common indices
    filtered_indices_base = torch.load(filtered_indices_base_path)
    filtered_indices_base = [x.item() for x in filtered_indices_base]
    curr_flipped_indices = set(filtered_indices_base)
    curr_unique_flipped_indices = curr_flipped_indices.difference(prev_flipped_indices)    
    prev_flipped_indices.update(curr_unique_flipped_indices)
    curr_unique_flipped_indices = list(curr_unique_flipped_indices)
    all_flipped_indices.append(filtered_indices_base)

    with open(images_cw_base, 'rb') as in_file:
        images_cw_base = pickle.load(in_file)
        images_cw_base[0] = images_cw_base[0][curr_unique_flipped_indices]
        images_cw_base[1] = images_cw_base[1][curr_unique_flipped_indices]
        
    with open(images_benign, 'rb') as in_file:
        images_benign = pickle.load(in_file)
        images_benign[0] = images_benign[0][curr_unique_flipped_indices]
        images_benign[1] = images_benign[1][curr_unique_flipped_indices]

    with open(predictions_base_path, 'rb') as in_file:
        predictions = pickle.load(in_file)
        true_labels_base = np.asarray(predictions['true_labels'])[curr_unique_flipped_indices]
        pred_labels_base = np.asarray(predictions['pred_labels'])[curr_unique_flipped_indices]

    save_path = os.path.join(base_path, 'Visualizations')
    

    for idx in range(10):
        benign = images_benign[0][idx]
        cw_base = images_cw_base[0][idx]

        benign = benign.swapaxes(0,1)
        benign = benign.swapaxes(1,2)

        cw_base = cw_base.swapaxes(0,1)
        cw_base = cw_base.swapaxes(1,2)


        title_benign = f'GT:{true_labels_base[idx]}'
        title_base = f'Cls:{pred_labels_base[idx]}'
        titles = [title_benign, title_base]
        plot_side_by_side(benign, cw_base,  idx, save_path, titles=titles)

In [ ]:
og_num_samples = 2500
all_freq_lists = []
all_theoretical_eps_per_channel = []
all_mse_loss_per_channel = []
all_mse_loss_per_pixel = []
all_mse_max_per_pixel = []
all_mse_loss_per_image = []
    
for eps_idx, base_path in enumerate(all_base_paths[3:4]):
    num_samples = 2500
    c_base=1.0
    all_flipped_indices = []
    lst_theoretical_eps_per_channel = []
    lst_mse_loss_per_channel = []
    lst_mse_loss_per_pixel = []
    lst_mse_max_per_pixel = []
    lst_mse_loss_per_image = []

    prev_flipped_indices = set()
    eps = all_eps_list[eps_idx][-1]
    images_cw_base = f"Adversarial_Datasets/CW_adv_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    images_benign = f"Benign_Datasets/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    predictions_base = f"Predictions/Model/CW_type-adversarial_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pickle"
    filtered_indices_base = f"Predictions/Perturbed_Samples/CW_benign_samples_{num_samples}_test_detector-type-{detector}_integrated-False_c-{c_base}_d-{d_base}_eps-{eps}.pt"
    images_cw_base = os.path.join(base_path, images_cw_base)
    images_benign = os.path.join(base_path, images_benign)
    predictions_base_path = os.path.join(base_path, predictions_base)
    filtered_indices_base_path = os.path.join(base_path, filtered_indices_base)

    # Filter by common indices
    filtered_indices_base = torch.load(filtered_indices_base_path)
    filtered_indices_base = [x.item() for x in filtered_indices_base]
    curr_flipped_indices = set(filtered_indices_base)
    curr_unique_flipped_indices = curr_flipped_indices.difference(prev_flipped_indices)    
    prev_flipped_indices.update(curr_unique_flipped_indices)
    curr_unique_flipped_indices = list(curr_unique_flipped_indices)
    all_flipped_indices.append(filtered_indices_base)

    with open(images_cw_base, 'rb') as in_file:
        images_cw_base = pickle.load(in_file)
        images_cw_base[0] = images_cw_base[0][curr_unique_flipped_indices]
        images_cw_base[1] = images_cw_base[1][curr_unique_flipped_indices]
        
    with open(images_benign, 'rb') as in_file:
        images_benign = pickle.load(in_file)
        images_benign[0] = images_benign[0][curr_unique_flipped_indices]
        images_benign[1] = images_benign[1][curr_unique_flipped_indices]

    with open(predictions_base_path, 'rb') as in_file:
        predictions = pickle.load(in_file)
        true_labels_base = np.asarray(predictions['true_labels'])[curr_unique_flipped_indices]
        pred_labels_base = np.asarray(predictions['pred_labels'])[curr_unique_flipped_indices]

    save_path = os.path.join(base_path, 'Visualizations')
    

    for idx in range(10):
        benign = images_benign[0][idx]
        cw_base = images_cw_base[0][idx]

        benign = benign.swapaxes(0,1)
        benign = benign.swapaxes(1,2)

        cw_base = cw_base.swapaxes(0,1)
        cw_base = cw_base.swapaxes(1,2)


        title_benign = f'GT:{true_labels_base[idx]}'
        title_base = f'Cls:{pred_labels_base[idx]}'
        titles = [title_benign, title_base]
        plot_side_by_side(benign, cw_base,  idx, save_path, titles=titles)

In [ ]:
benign.shape